In [ ]:
"""
Get data of actors from IMDB page.
Import movie_data for actor and link_s and scrape the actors pages.
For each actor, find list of movies and scrape movie pages for brief info.

functions used:
get_soup(url) from scrape.py
get_movie_brief(soup) from get_movie.py

Output to pickle 'actor_data'.

"""

In [ ]:
from bs4 import BeautifulSoup
import requests

import pandas as pd
import numpy as np

import pickle

import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_2/py')
from scrape import *
from get_movie import *

In [ ]:
# Load 'movie'
movie_df = pd.read_pickle('../data/movie_data_adaptation')

In [ ]:
# Create list of actor/link from the 2000 movies
star_list = []
for star in movie_df.star:
    star_list += star
star_df = pd.DataFrame(star_list)

star_link_list = []
for star_link in movie_df.link_s:
    star_link_list += star_link
star_link_df = pd.DataFrame(star_link_list)

# Combine in to one df
star_df['link_s'] = star_link_df
star_df.columns = ['star','link_s']
star_df = star_df.drop_duplicates(subset=['star'])
star_df

In [ ]:
# Create actor_link_dict
actor_link_dict = star_df.set_index('star').T.to_dict('list')

In [ ]:
# Create list of movie_dict for each actor
actor_film_list = []

base_url = "https://www.imdb.com"
c = 0 # to check scraping progress

for actor in actor_link_dict:
    c += 1
    print(c, actor)
    actor_url = base_url + actor_link_dict[actor][0]
    actor_soup = get_soup(actor_url)
    
    # actor or actress
    actor_head = actor_soup.find('div', id="filmo-head-actor")
    if actor_head is None:
        actor_head = actor_soup.find('div', id="filmo-head-actress")
    
    films = actor_head.findNext('div')

    for film in films:
        movie = film.find('a')
        if type(movie) != int:
            movie_title = movie.text
            movie_link = movie.get('href')
            movie_soup = get_soup(base_url + movie_link)

            detail = get_movie_brief_actor(actor,movie_soup)
            actor_film_list.append(detail)

In [ ]:
# Create actor_df and save pickle as 'actor_data'
actor_df = pd.DataFrame(actor_film_list)
actor_df.to_pickle('../data/actor_data')